# Create a Ryu OpenFlow Controller

#### Configure the Environment

In [ ]:
# Set project and region.
export OS_REGION_NAME=CHI@UC
export OS_PROJECT_NAME=CH-822154

# Define networking variables.
NODE_TYPE=compute_haswell
KEYPAIR_NAME="my_chameleon_key"
KEYPAIR_LOCATION="/home/"${USER}"/work/exercises/"${KEYPAIR_NAME}".pem" # 
LEASE_NAME="${USER}RyuControllerLease"
SERVER_NAME="${USER}RyuController"

#### Create Lease

In [ ]:
# Obtain public network id.
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

# Reserve a node and a floating IP on public network.
blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M' -d'+1 minute')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

#### Start the Server

In [ ]:
# Create a new server using your reservation
openstack server create --max-width 80 \
                        --key-name $KEYPAIR_NAME \
                        $(lease_server_create_default_args "$LEASE_NAME") \
                        $SERVER_NAME

#### Associate the Floating IP

In [ ]:
# Find the unused floating IP you reserved
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

# Add your floating IP to your server
openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

#### Run Script

This script observes your node using TCP access.

In [ ]:
RYU_APP='simple_switch_13_custom_chameleon.py'
RYU_PORT='6653'
MIRROR_PORT=''

{   
yum install -y yum-utils device-mapper-persistent-data lvm2 vim   
yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo   
yum install -y docker-ce   
systemctl start docker   
   
RECIPE_REPO=\"https://github.com/RENCI-NRIG/exogeni-recipes.git\"   
RECIPE_DIR=\"/opt/exogeni-recipes\"   
RECIPE_APP=\"openflow-controller/docker\"   
DOCKER_IMAGE=\"centos-ryu\"   
DOCKER_CONTAINER_NAME=\"ryu-controller\"   
OFP_TCP_LISTEN_PORT=\"6653\"   
RYU_APP=\"/opt/ryu_app/simple_switch_13_custom_chameleon.py\"   
MIRROR_PORT=''
   
git clone  --no-checkout \${RECIPE_REPO} \${RECIPE_DIR}   
cd \${RECIPE_DIR} && git config core.sparsecheckout true   
echo \"\${RECIPE_APP}/*\" >> .git/info/sparse-checkout   
git read-tree -m -u HEAD   
   
cd \${RECIPE_DIR}/\${RECIPE_APP}   
docker volume create var_run_ryu   
docker volume create var_log_ryu   
docker volume create opt_ryu   
docker volume create opt_ryu_chameleon   
"sed -r -i \s/^(RYU_APP=.*)/#\1/g\ ryu_start.sh"   
"sed -r -i \s/^(OFP_TCP_LISTEN_PORT=.*)/#\1/g\ ryu_start.sh"   
   
docker build -t \${DOCKER_IMAGE} .   
   
docker run --rm -dit -p \${OFP_TCP_LISTEN_PORT}:\${OFP_TCP_LISTEN_PORT} -p 8080:8080 -v opt_ryu_chameleon:/opt/ryu_chameleon -v opt_ryu:/opt/ryu -v var_log_ryu:/var/log/ryu -v var_run_ryu:/var/run/ryu -e RYU_APP=\${RYU_APP} -e OFP_TCP_LISTEN_PORT=\${OFP_TCP_LISTEN_PORT}  --name=\${DOCKER_CONTAINER_NAME} \${DOCKER_IMAGE}   
   
echo done   
   
} > /tmp/boot.log 2>&1

cat /tmp/boot.log

#### Clean up

In [ ]:
openstack server delete $SERVER_NAME
blazar lease-delete $LEASE_NAME